In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
API_KEY_ALPHA_VANTAGE = ""

In [ ]:
def collect_news_data(query, api_key=API_KEY_ALPHA_VANTAGE):
    """Collect news headlines data from Alpha Vantage News API."""
    url = f"https://www.alphavantage.co/query?{query}" +f"&apikey={api_key}"
    response = requests.get(url)
    news_data = response.json()
    headlines = [[item[key] for key in ["title", "url", "time_published", "source", "overall_sentiment_score", "overall_sentiment_label", "ticker_sentiment"]] for item in news_data.get("feed", [])]
    df = pd.DataFrame(headlines, columns=["title", "url", "time_published", "source", "overall_sentiment_score", "overall_sentiment_label", "ticker_sentiment"])
    return df

In [ ]:
def get_data_from_date_range(start_time, end_time, saveFileName):
  # Convert the string timestamps to pandas datetime objects
  start_date = pd.to_datetime(start_time, format="%Y%m%dT%H%M")
  end_date = pd.to_datetime(end_time, format="%Y%m%dT%H%M")

  # Create a list to hold the date ranges
  date_ranges = []

  # Iterate from start_date to end_date in increments of 6 days
  current_date = start_date
  while current_date < end_date:
      # Define the end date for the current interval
      interval_end = current_date + pd.DateOffset(days=1)  # 6 days later (inclusive)

      # Append the formatted date range to the list
      date_ranges.append((current_date.strftime('%Y%m%dT0000'), interval_end.strftime('%Y%m%dT2359')))

      # Move to the next interval
      current_date += pd.DateOffset(days=1)

  # Display the results
  print(date_ranges)
  print(len(date_ranges))

  #Call apis
  dfs = []

  for start, end in date_ranges:
    df = collect_news_data(f"function=NEWS_SENTIMENT&topics=technology&time_from={start}&time_to={end}&sort=LATEST&limit=50")
    dfs.append(df)

  merged_df = pd.concat(dfs, ignore_index=True)

  merged_df['time_published'] = pd.to_datetime(merged_df['time_published'])

  merged_df = merged_df.sort_values(by='time_published')

  merged_df.to_csv(saveFileName, index=False)

  print(len(merged_df))
  display(merged_df)
  return merged_df

In [ ]:
#Combine file fragments into 1 dataframe
allFiles = []

for i in range(1,5):
  allFiles.append(pd.read_csv(f'technology_news_data_future_{i}.csv'))

merged_vertical = pd.concat(allFiles, ignore_index=True)
merged_vertical.to_csv('merged_technology_news_data_future_3months.csv', index=False)